In [ ]:
# -*- coding: utf-8
# Reinaldo Chaves (reichaves@gmail.com)

O projeto que implementa um sistema de análise de dados de processos minerários brasileiros.

## Objetivo Principal
O script analisa dados do **SIGMINE** (Sistema de Informações Geográficas da Mineração) da ANM (Agência Nacional de Mineração), que contém informações sobre processos minerários ativos no Brasil.

## Funcionalidades Principais

### 1. **Análise Estatística de Processos Minerários**
- Calcula estatísticas gerais: total de feições, área total em hectares, área mediana
- Identifica os 10 maiores processos por área
- Distribui dados por UF (Unidade Federativa) e por FASE do processo

### 2. **Integração de Dados Geoespaciais e Administrativos**
- Lê arquivos shapefile (.shp) com dados geográficos dos processos
- Integra com dados do Sistema de Cadastro Mineiro (SCM) para obter informações detalhadas sobre:
  - Pessoas físicas/jurídicas responsáveis (CPF/CNPJ)
  - Municípios onde os processos estão localizados
  - Substâncias minerais envolvidas

### 3. **Correção de Geometrias**
- Aplica correções em geometrias inválidas usando `make_valid()` do GeoPandas
- Garante que todas as features sejam válidas para análise espacial

### 4. **Geração de Fichas Detalhadas**
- Cria fichas completas de processos específicos contendo:
  - Número do processo
  - Nome e documento (CPF/CNPJ) do responsável
  - Tipo de pessoa (física/jurídica)
  - Município e UF
  - Substância mineral
  - Fase do processo
  - Área em hectares

### 5. **Exportação de Resultados**
- Salva fichas de processos em formato CSV
- Organiza resultados em diretório específico

## Fontes de Dados
- **SIGMINE**: dados geográficos dos processos minerários
- **Sistema de Cadastro Mineiro**: dados administrativos e cadastrais
- **IBGE**: códigos e nomes dos municípios brasileiros

## Tecnologias Utilizadas
- **GeoPandas**: manipulação de dados geoespaciais
- **Pandas**: análise de dados tabulares
- **Shapely**: operações geométricas
- **Google Colab**: ambiente de execução

Todas as informações precisam ser checadas

In [ ]:
# -----------------------------------------------
# 📊  Estatísticas de processos minerários (ANM)
#     Notebook para Google Colab
# -----------------------------------------------

In [ ]:
'''
Fonte dos dados:
https://dados.gov.br/dados/conjuntos-dados/sistema-de-informacoes-geograficas-da-mineracao-sigmine
Arquivo de Metadados
Processos minerários ativos - Brasil

+ Cadastro Mineiro
https://dados.gov.br/dados/conjuntos-dados/sistema-de-cadastro-mineiro
Microdados SCM
https://app.anm.gov.br/dadosabertos/SCM/microdados/
'''

In [ ]:
# 1) Instalar dependências ──────────────────────
# (Shapely 2+ é compatível com GeoPandas ≥ 0.14)
!pip -q install geopandas shapely pyproj fiona pyogrio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 24.4 MB/s eta 0:00:00


In [ ]:
# 2) Montar o Google Drive ──────────────────────
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 3) Definir pasta dos dados ────────────────────
base_dir = "/content/drive/My Drive/domingo_de_dados_2025/dados_ambientais/sigmine_ativos/dados/BRASIL"

In [ ]:
# 4) Localizar o arquivo .shp ───────────────────
import glob, os
shp_files = glob.glob(os.path.join(base_dir, "*.shp"))
assert shp_files, "Nenhum .shp encontrado no caminho especificado."
shp_path = shp_files[0]
print(f"Usando shapefile: {shp_path}")

Usando shapefile: /content/drive/My Drive/domingo_de_dados_2025/dados_ambientais/sigmine_ativos/dados/BRASIL/BRASIL.shp


In [ ]:
# 5) Ler com GeoPandas ──────────────────────────
import geopandas as gpd, pandas as pd

gdf = gpd.read_file(shp_path)

# se precisar corrigir de verdade - Para garantir que todas as features sejam válidas WKT
gdf["geometry"] = gdf.geometry.make_valid()        # GeoPandas >= 0.14 / Shapely >= 2
# alternativa tradicional (buffer‑0) – menos robusta
# gdf["geometry"] = gdf.geometry.buffer(0)



/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:198: RuntimeWarning: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
  return ogr_read(


In [ ]:
# 6) Estatísticas principais ─────────────────────
n_feat   = len(gdf)
total_ha = gdf["AREA_HA"].sum()
med_ha   = gdf["AREA_HA"].median()

In [ ]:
print(f"Total de feições: {n_feat:,}")
print(f"Área total: {total_ha:,.0f} ha")
print(f"Área mediana: {med_ha:,.1f} ha")

Total de feições: 255,230
Área total: 218,241,541 ha
Área mediana: 74.0 ha


In [ ]:
# 7) Top‑10 maiores áreas ───────────────────────
top10 = gdf.nlargest(10, "AREA_HA")[["DSProcesso", "UF", "AREA_HA"]]
display(top10)

,DSProcesso,UF,AREA_HA
181654,803.237/2022,PI,207815.13
192323,850.232/2023,PA,167348.57
227198,813.684/1969,PA,98910.42
66744,605.626/1976,SE,92498.50
543,001.662/1936,MG,74633.97
198667,800.988/2023,CE,48914.66
684,813.682/1969,PA,30000.00
237514,815.118/1994,SC,26385.82
293,002.019/1939,GO,24542.04
162912,886.166/2021,RO,20882.67


In [ ]:
registro = gdf.query("PROCESSO == '803237/2022'")     # ou DSProcesso == '803237/2022'
print(registro.T)           # exibe todas as colunas verticalmente

                                                       181654
PROCESSO                                          803237/2022
NUMERO                                                 803237
ANO                                                      2022
AREA_HA                                             207815.13
ID                     {923B1CA9-6840-49DA-99C9-82A08CB0DD27}
FASE                                 RECONHECIMENTO GEOLÓGICO
ULT_EVENTO  10 - RECONH GEOL/REQUERIMENTO RECONHECIMENTO G...
NOME           AVANT GEOTECNOLOGIAS E ENGENHARIA MINERAL LTDA
SUBS                                      DADO NÃO CADASTRADO
USO                                       DADO NÃO CADASTRADO
UF                                                         PI
DSProcesso                                       803.237/2022
geometry    POLYGON Z ((-42.077533332999906 -4.16381388899...


In [ ]:
# 8) Distribuição por UF e por FASE ─────────────
uf_counts   = gdf["UF"].value_counts().rename_axis("UF").to_frame("feições")
fase_counts = gdf["FASE"].value_counts().rename_axis("FASE").to_frame("feições")

In [ ]:
display(uf_counts.head())
display(fase_counts.head())

,feições
UF,
MG,49523
BA,29039
PA,23914
SP,15732
GO,15196


,feições
FASE,
AUTORIZAÇÃO DE PESQUISA,104091
REQUERIMENTO DE PESQUISA,24405
REQUERIMENTO DE LAVRA,21449
LICENCIAMENTO,20658
DISPONIBILIDADE,20304


### PARA VER O CPF OU CNPJ DO RESPONSÁVEL

In [ ]:
!pip -q install geopandas shapely pyogrio pandas pyarrow openpyxl

In [ ]:
import os, csv, glob, zipfile, re, pandas as pd, geopandas as gpd

In [ ]:
# ── 0. caminhos --------------------------------------------------------------
BASE  = "/content/drive/My Drive/domingo_de_dados_2025/dados_ambientais/sigmine_ativos/dados"
SHP   = f"{BASE}/BRASIL/BRASIL.shp"
TXT   = f"{BASE}/microdados-scm"

files = dict(
    pessoa   = f"{TXT}/Pessoa.txt",
    proc_pes = f"{TXT}/ProcessoPessoa.txt",
    proc_mun = f"{TXT}/ProcessoMunicipio.txt",
    proc_sub = f"{TXT}/ProcessoSubstancia.txt",
    subst    = f"{TXT}/Substancia.txt",
)

In [ ]:
# ── 1. utilidades ------------------------------------------------------------
norm = lambda s: re.sub(r"[^A-Z0-9]", "", str(s).upper())
read = lambda p, cols=None: pd.read_csv(
    p, sep=";", encoding="latin1", engine="python", on_bad_lines="warn",
    usecols=cols, dtype=str)

In [ ]:
# ── 2. leitura e normalização -----------------------------------------------
pessoa   = read(files["pessoa"],
                lambda c: norm(c) in {"IDPESSOA","NRCPFCNPJ","TPPESSOA","NMPESSOA"})
pessoa.columns = [norm(c) for c in pessoa.columns]

proc_pes = read(files["proc_pes"],
                lambda c: norm(c) in {"DSPROCESSO","IDPESSOA"})
proc_pes.columns = [norm(c) for c in proc_pes.columns]

proc_mun = read(files["proc_mun"],
                lambda c: norm(c) in {"DSPROCESSO","IDMUNICIPIO"})
proc_mun.columns = [norm(c) for c in proc_mun.columns]

proc_sub = read(files["proc_sub"],
                lambda c: norm(c) in {"DSPROCESSO","IDSUBSTANCIA"})
proc_sub.columns = [norm(c) for c in proc_sub.columns]

subst    = read(files["subst"],
                lambda c: norm(c) in {"IDSUBSTANCIA","NMSUBSTANCIA"})
subst.columns = [norm(c) for c in subst.columns]

In [ ]:
# ── 3. juntar pessoa ↔ processo ---------------------------------------------
scm = proc_pes.merge(pessoa, on="IDPESSOA", how="left")

# ── 4. anexar município ------------------------------------------------------
scm = scm.merge(proc_mun, on="DSPROCESSO", how="left")

In [ ]:
# (opcional) traduzir código‑IBGE em nome
ibge_url = "https://raw.githubusercontent.com/kelvins/municipios-brasileiros/master/csv/municipios.csv"
ibge = pd.read_csv(ibge_url, dtype=str)
ibge = ibge.rename(columns={'codigo_ibge': 'IDMUNICIPIO'})
ibge = ibge.rename(columns={'nome': 'nome_municipio'})
scm = scm.merge(ibge, on="IDMUNICIPIO", how="left")

In [ ]:
ibge.columns

Index(['IDMUNICIPIO', 'nome_municipio', 'latitude', 'longitude', 'capital',
       'codigo_uf', 'siafi_id', 'ddd', 'fuso_horario'],
      dtype='object')

In [ ]:
# ── 5. anexar substância -----------------------------------------------------
scm = scm.merge(proc_sub, on="DSPROCESSO", how="left")
scm = scm.merge(subst,    on="IDSUBSTANCIA", how="left")

In [ ]:
# ── 6. shapefile SIGMINE -----------------------------------------------------
gdf = gpd.read_file(SHP)[["DSProcesso","UF","FASE","AREA_HA","geometry"]]
full = gdf.merge(scm, left_on="DSProcesso", right_on="DSPROCESSO", how="left")

/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Polygon' is converted to 'Polygon Z'
  return ogr_read(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:198: RuntimeWarning: Non closed ring detected. To avoid accepting it, set the OGR_GEOMETRY_ACCEPT_UNCLOSED_RING configuration option to NO
  return ogr_read(


In [ ]:
# ── 7. ficha do processo -----------------------------------------------------
proc = "803.237/2022"
cols = ["DSProcesso","NMPESSOA","NRCPFCNPJ","TPPESSOA",
        "nome_municipio","UF","NMSUBSTANCIA","FASE","AREA_HA"]
print(full.query("DSProcesso == @proc")[cols].T)

                                  948417                    948418  \
DSProcesso                  803.237/2022              803.237/2022   
NMPESSOA         DANIEL FERREIRA MARIANO   DANIEL FERREIRA MARIANO   
NRCPFCNPJ                    ***699106**               ***699106**   
TPPESSOA                               F                         F   
nome_municipio                    Barras                   Batalha   
UF                                    PI                        PI   
NMSUBSTANCIA                         NaN                       NaN   
FASE            RECONHECIMENTO GEOLÓGICO  RECONHECIMENTO GEOLÓGICO   
AREA_HA                        207815.13                 207815.13   

                                  948419                    948420  \
DSProcesso                  803.237/2022              803.237/2022   
NMPESSOA         DANIEL FERREIRA MARIANO   DANIEL FERREIRA MARIANO   
NRCPFCNPJ                    ***699106**               ***699106**   
TPPESSOA           

In [ ]:
# ── 8. salvar ficha em CSV ───────────────────────────────────
import os

# 1) definir diretório‑alvo
out_dir = "/content/drive/My Drive/domingo_de_dados_2025/dados_ambientais/sigmine_ativos/dados/resultados"
os.makedirs(out_dir, exist_ok=True)          # cria se não existir

# 2) filtrar novamente (caso mude o proc) e salvar
proc = "803.237/2022"                        # <- use o mesmo formato de DSProcesso
cols = ["DSProcesso","NMPESSOA","NRCPFCNPJ","TPPESSOA",
        "nome_municipio","UF","NMSUBSTANCIA","FASE","AREA_HA"]

registro = full.query("DSProcesso == @proc")[cols]
fname    = os.path.join(out_dir, f"ficha_{proc.replace('/','_').replace('.','')}.csv")

registro.to_csv(fname, index=False, encoding="utf-8")
print(f"✔ CSV salvo em: {fname}")

✔ CSV salvo em: /content/drive/My Drive/domingo_de_dados_2025/dados_ambientais/sigmine_ativos/dados/resultados/ficha_803237_2022.csv
